In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sklearn
from tqdm import tqdm

import data_interface
import mnar_blackout_lds

In [3]:
# Load the data
x_t, m_t, meta = data_interface.load_panel()

In [4]:
model_params = mnar_blackout_lds.MNARParams.init_random(K=9, D=147, seed=42)
model = mnar_blackout_lds.MNARBlackoutLDS(model_params)

In [5]:
em_train_history = model.em_train(x_t, m_t, num_iters=10)


=== EM iteration 1/10 ===
  A norm: 2.963
  Q trace: 4.938
  mean diag(R): 58.312

=== EM iteration 2/10 ===
  A norm: 2.843
  Q trace: 20.078
  mean diag(R): 30.773

=== EM iteration 3/10 ===
  A norm: 2.854
  Q trace: 28.317
  mean diag(R): 28.077

=== EM iteration 4/10 ===
  A norm: 2.860
  Q trace: 33.414
  mean diag(R): 27.765

=== EM iteration 5/10 ===
  A norm: 2.866
  Q trace: 37.171
  mean diag(R): 27.751

=== EM iteration 6/10 ===
  A norm: 2.870
  Q trace: 40.327
  mean diag(R): 27.797

=== EM iteration 7/10 ===
  A norm: 2.872
  Q trace: 43.029
  mean diag(R): 27.832

=== EM iteration 8/10 ===
  A norm: 2.874
  Q trace: 45.008
  mean diag(R): 27.840

=== EM iteration 9/10 ===
  A norm: 2.875
  Q trace: 46.251
  mean diag(R): 27.835

=== EM iteration 10/10 ===
  A norm: 2.877
  Q trace: 46.970
  mean diag(R): 27.822


### Reconstruction and Prediction

In [6]:
ekf_out = model.ekf_forward(x_t, m_t)
smoother_out = model.rts_smoother(ekf_out)

mu_filt = ekf_out["mu_filt"]
Sigma_filt = ekf_out["Sigma_filt"]
mu_smooth = smoother_out["mu_smooth"]
Sigma_smooth = smoother_out["Sigma_smooth"]

In [7]:
evaluation_windows = data_interface.get_eval_windows("data")

In [8]:
# Evaluate imputation performance

impute_mae_list = []
impute_mse_list = []

for window in tqdm(evaluation_windows):
    if window["test_type"] == "impute":
        start_idx = np.where(meta["timestamps"]==window["blackout_start"])[0][0]
        end_idx = np.where(meta["timestamps"]==window["blackout_end"])[0][0]

        detector_idx = np.where(meta["detectors"]==window["detector_id"])[0][0]

        eval_x_t = x_t[start_idx:end_idx+1].copy()
        eval_mu_smooth = mu_smooth[start_idx:end_idx+1]
        eval_Sigma_smooth = Sigma_smooth[start_idx:end_idx+1]

        reconstruct_x_t, _ = model.reconstruct_from_smoother(eval_mu_smooth, eval_Sigma_smooth)

        mae = sklearn.metrics.mean_absolute_error(reconstruct_x_t[:, detector_idx], eval_x_t[:, detector_idx])
        mse = sklearn.metrics.mean_squared_error(reconstruct_x_t[:, detector_idx], eval_x_t[:, detector_idx])
        rmse = np.sqrt(mse)
        
        impute_mae_list.append([mae, window["len_steps"]])
        impute_mse_list.append([mse, window["len_steps"]])

final_mae = np.average([item[0] for item in impute_mae_list], weights=[item[1] for item in impute_mae_list])
final_mse = np.average([item[0] for item in impute_mse_list], weights=[item[1] for item in impute_mse_list])
final_rmse = np.sqrt(final_mse)

print("Final MAE (impute):", final_mae)
print("Final MSE (impute):", final_mse)
print("Final RMSE (impute):", final_rmse)

100%|██████████| 1764/1764 [00:00<00:00, 4692.34it/s]

Final MAE (impute): 1.6935249175996532
Final MSE (impute): 5.645882503504708
Final RMSE (impute): 2.376106585047209


In [9]:
# Evaluate forecasting performance

y_actual_1_step, y_forecast_1_step = [], []
y_actual_3_step, y_forecast_3_step = [], []
y_actual_6_step, y_forecast_6_step = [], []

for window in tqdm(evaluation_windows):
    if window["test_type"] == "forecast":
        start_idx = np.where(meta["timestamps"]==window["blackout_start"])[0][0]
        blackout_idx = start_idx - 1

        end_idx = np.where(meta["timestamps"]==window["blackout_end"])[0][0]

        detector_idx = np.where(meta["detectors"]==window["detector_id"])[0][0]

        eval_x_t = x_t[start_idx:end_idx+1].copy()

        forecast_x_t, _ = model.k_step_forecast(mu_filt, Sigma_filt, blackout_idx, k=int(window["horizon_steps"]))
        
        if int(window["horizon_steps"]) == 1:
            y_forecast_1_step.append(forecast_x_t[detector_idx])
            y_actual_1_step.append(eval_x_t[int(window["horizon_steps"])-1, detector_idx])
        elif int(window["horizon_steps"]) == 3:
            y_forecast_3_step.append(forecast_x_t[detector_idx])
            y_actual_3_step.append(eval_x_t[int(window["horizon_steps"])-1, detector_idx])
        elif int(window["horizon_steps"]) == 6:
            y_forecast_6_step.append(forecast_x_t[detector_idx])
            y_actual_6_step.append(eval_x_t[int(window["horizon_steps"])-1, detector_idx])

mae_1_step = sklearn.metrics.mean_absolute_error(y_forecast_1_step, y_actual_1_step)
mse_1_step = sklearn.metrics.mean_squared_error(y_forecast_1_step, y_actual_1_step)
rmse_1_step = np.sqrt(mse_1_step)

print("Evaluation results for forecasting:")
print("\n-----------------------------------")
print("1-step MAE (forecast):", mae_1_step)
print("1-step MSE (forecast):", mse_1_step)
print("1-step RMSE (forecast):", rmse_1_step)


mae_3_step = sklearn.metrics.mean_absolute_error(y_forecast_3_step, y_actual_3_step)
mse_3_step = sklearn.metrics.mean_squared_error(y_forecast_3_step, y_actual_3_step)
rmse_3_step = np.sqrt(mse_3_step)
print("\n-----------------------------------")
print("3-step MAE (forecast):", mae_3_step)
print("3-step MSE (forecast):", mse_3_step)
print("3-step RMSE (forecast):", rmse_3_step)

mae_6_step = sklearn.metrics.mean_absolute_error(y_forecast_6_step, y_actual_6_step)
mse_6_step = sklearn.metrics.mean_squared_error(y_forecast_6_step, y_actual_6_step)
rmse_6_step = np.sqrt(mse_6_step)
print("\n-----------------------------------")
print("6-step MAE (forecast):", mae_6_step)
print("6-step MSE (forecast):", mse_6_step)
print("6-step RMSE (forecast):", rmse_6_step)

100%|██████████| 1764/1764 [00:00<00:00, 10201.31it/s]

Evaluation results for forecasting:

-----------------------------------
1-step MAE (forecast): 2.1523494370570853
1-step MSE (forecast): 8.918860825804575
1-step RMSE (forecast): 2.986446186658078

-----------------------------------
3-step MAE (forecast): 1.6765516351880905
3-step MSE (forecast): 4.761804266464367
3-step RMSE (forecast): 2.1821558758403046

-----------------------------------
6-step MAE (forecast): 1.490643806003054
6-step MSE (forecast): 4.163694755538252
6-step RMSE (forecast): 2.0405133558833306


In [10]:
len(impute_mae_list), len(y_actual_1_step), len(y_actual_3_step), len(y_actual_6_step)

(441, 441, 441, 441)